In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Useful sklearn imports
from sklearn.linear_model import LinearRegression, PoissonRegressor
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)

# Poisson regression

## Data loading and exploration

In [ ]:
def real_data_poisson():
    # Load the species data
    try:
        especies = pd.read_csv(
            'especies.txt',
            sep="\t"
        )
    except Exception as e:
        print(f"Error loading Expecies dataset: {e}.")

    # Features
    X_especies = especies.drop('Especies', axis=1)
    # Target variable
    y_especies = especies['Especies']

    return X_especies, y_especies

### Get real data for poisson regression

In [ ]:
X, y = real_data_poisson()

### Explore the data, visualizing it, etc.

In [ ]:
# What type of data do we have?
print('X data types:')
print(X.dtypes)
print('\n')
print('y data types:')
print(y.dtypes)

In [ ]:
X.head()

### Plot the data

In [ ]:
# 3d plot, with X features as x and y, and y as z
# We will be using matplotlib's 3d plotting capabilities
plt.figure(figsize=(10, 6))
ax = plt.axes(projection='3d')
ax.scatter(X['pH'], X['Biomasa'], y, c=y, cmap='viridis', linewidth=0.5)
ax.set_xlabel('pH')
ax.set_ylabel('Biomasa')
ax.set_zlabel('Especies')
plt.title('Species Data')
plt.show()

In [ ]:
# Per-PH value plot
plt.figure(figsize=(8,6))
colors = {0: 'blue', 1: 'red', 2: 'green'}
markers = {0: 'o', 1: 's', 2: '^'}
# Select the subset of the data for each pH value
for ph_value in X['pH'].unique():
    x_subset = X[X['pH'] == ph_value]['Biomasa']
    y_subset = y.loc[x_subset.index]
    plt.scatter(
        x_subset,
        y_subset,
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value}'
    )

## Fitting Poisson Regression models

### Fit linear regression models, per pH value

In [ ]:
# We  keep track of the models, to plot them later
linear_1d_models = {}

for ph_value in X['pH'].unique():
    # Subset the data
    x_subset = X[X['pH'] == ph_value]['Biomasa']
    y_subset = y.loc[x_subset.index]

    # Define the Linear Regression model, for this ph value
    model = LinearRegression()
    # Fit the model
    model.fit(
        ???
    )
    # Save the model
    linear_1d_models[ph_value] = model

    # Predict the values
    x_vals = np.linspace(0, 10, 100).reshape(-1, 1)
    y_vals = model.predict(x_vals)
    
    # Plot the line
    plt.plot(x_vals, y_vals, color=colors[ph_value])
    # As well as the original data
    plt.scatter(
        x_subset,
        y_subset,
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value}'
    )
    # Plot a line in the origin
    plt.axhline(0, color='black', linewidth=0.5)
plt.xlabel("Biomasa")
plt.ylabel("Especies")
plt.legend()
plt.title("Linear regressors of Biomasa vs. Especies, per pH value")
plt.show()

### Fit Poisson regression models, per pH value

In [ ]:
# We  keep track of the models, to plot them later
poisson_1d_models = {}

for ph_value in X['pH'].unique():
    # Subset the data
    x_subset = X[X['pH'] == ph_value]['Biomasa']
    y_subset = y.loc[x_subset.index]

    # Define the Poisson Regression model
    model = PoissonRegressor()
    # Fit the model
    model.fit(
        ???
    )
    # Save the model
    poisson_1d_models[ph_value] = model
    
    # Predict the values
    x_vals = np.linspace(0, 10, 100).reshape(-1, 1)
    y_vals = model.predict(x_vals)
    # Plot the line
    plt.plot(x_vals, y_vals, color=colors[ph_value])
    # As well as the original data
    plt.scatter(
        x_subset,
        y_subset,
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value}'
    )
    # Plot a line in the origin
    plt.axhline(0, color='black', linewidth=0.5)

plt.xlabel("Biomasa")
plt.ylabel("Especies")
plt.legend()
plt.title("Poisson regressors of Biomasa vs. Especies, per pH value")
plt.show()

### Fit a single Poisson regression model

In [ ]:
# Train a Poisson Regression model over all features
poisson_full_model = PoissonRegressor()
poisson_full_model.fit(
    ???
)

### Evaluation of models

In [ ]:
# Plot the original data, the predictions of each of the 1d Poisson models, and the full Poisson model, per pH value
plt.figure(figsize=(10, 6))
for ph_value in X['pH'].unique():
    x_subset = X[X['pH'] == ph_value]['Biomasa']
    y_subset = y.loc[x_subset.index]
    
    x_vals = np.linspace(0, 10, 100).reshape(-1, 1)
    # Using ph_value as color
    # Truth
    plt.scatter(
        x_subset, y_subset,
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value}'
    )
    # 1D model
    plt.scatter(
        x_subset,
        poisson_1d_models[ph_value].predict(x_subset.values.reshape(-1, 1)),
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value} (1D model predicted)',
        alpha=0.75
    )
    plt.plot(
        x_vals,
        poisson_1d_models[ph_value].predict(x_vals),
        color=colors[ph_value],
        label=f'pH {ph_value} (1D model predicted)',
        alpha=0.75
    )

    # Full model
    X_subset = X.loc[x_subset.index]
    plt.scatter(
        x_subset,
        poisson_full_model.predict(X_subset),
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value} (Full model predicted)',
        alpha=0.5
    )
    plt.plot(
        x_vals,
        poisson_full_model.predict(
            np.concatenate(
                (np.full((x_vals.shape[0], 1), ph_value), x_vals),
                axis=1
            )
        ),
        color=colors[ph_value],
        # Different line style
        linestyle='--',
        label=f'pH {ph_value} (Full model predicted)',
        alpha=0.5
    )
plt.legend()
plt.xlabel('Biomasa')
plt.ylabel('Especies')
plt.title('Poisson Regression Predictions, per pH value')
plt.show()

In [ ]:
# Replicate the above plot in log space:
# Plot the original log-data, the predictions of each of the 1d Poisson models (in log-space), and the full Poisson model predictions in log-space
plt.figure(figsize=(10, 6))
for ph_value in X['pH'].unique():
    x_subset = X[X['pH'] == ph_value]['Biomasa']
    y_subset = y.loc[x_subset.index]
    
    x_vals = np.linspace(0, 10, 100).reshape(-1, 1)
    # Using ph_value as color
    # Truth
    plt.scatter(
        x_subset, np.log(y_subset),
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value}'
    )
    # 1D model
    plt.scatter(
        x_subset,
        np.log(
            poisson_1d_models[ph_value].predict(x_subset.values.reshape(-1, 1))
        ),
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value} (1D model predicted)',
        alpha=0.75
    )
    plt.plot(
        x_vals,
        np.log(
            poisson_1d_models[ph_value].predict(x_vals)
        ),
        color=colors[ph_value],
        label=f'pH {ph_value} (1D model predicted)',
        alpha=0.75
    )

    # Full model
    X_subset = X.loc[x_subset.index]
    plt.scatter(
        x_subset,
        np.log(
            poisson_full_model.predict(X_subset)
        ),
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value} (Full model predicted)',
        alpha=0.5
    )
    plt.plot(
        x_vals,
        np.log(
            poisson_full_model.predict(
                np.concatenate(
                    (np.full((x_vals.shape[0], 1), ph_value), x_vals),
                    axis=1
                )
            )
        ),
        color=colors[ph_value],
        # Different line style
        linestyle='--',
        label=f'pH {ph_value} (Full model predicted)',
        alpha=0.5
    )
plt.legend()
plt.xlabel('Biomasa')
plt.ylabel('Especies')
plt.title('Poisson Regression Predictions, per pH value')
plt.show()

In [ ]:
# Compare the 1d and full poisson regression models

# Evaluate the 1d models
mae_1d = {}
mse_1d = {}
r2_1d = {}

for ph_value in X['pH'].unique():
    x_subset = X[X['pH'] == ph_value]['Biomasa']
    y_subset = y.loc[x_subset.index]
    y_pred_subset = poisson_1d_models[ph_value].predict(x_subset.values.reshape(-1, 1))
    mae_1d[ph_value] = mean_absolute_error(y_subset, y_pred_subset)
    mse_1d[ph_value] = mean_squared_error(y_subset, y_pred_subset)
    r2_1d[ph_value] = r2_score(y_subset, y_pred_subset)

# Evaluate the full model
y_pred_poisson = poisson_full_model.predict(X)
mae_full = mean_absolute_error(y, y_pred_poisson)
mse_full = mean_squared_error(y, y_pred_poisson)
r2_full = r2_score(y, y_pred_poisson)

# Print the results
print("\n--- Poisson Regression ---")
print("\n--------------------------")
print("1D Model Results:")
print("Mean Absolute Error: {}".format(
    np.mean(list(mae_1d.values())))
)
for ph_value, mae in mae_1d.items():
    print(f"\tpH {ph_value}: {mae}")
print("Mean Squared Error: {}".format(
    np.mean(list(mse_1d.values())))
)
for ph_value, mse in mse_1d.items():
    print(f"\tpH {ph_value}: {mse}")
print("R-squared: {}".format(
    np.mean(list(r2_1d.values())))
)
for ph_value, r2 in r2_1d.items():
    print(f"\tpH {ph_value}: {r2}")

print("\n--------------------------")
print("Full Model Results:")
print("Mean Absolute Error:", mae_full)
print("Mean Squared Error:", mse_full)
print("R-squared:", r2_full)


### Fit a Poisson regression model: with interaction terms

In [ ]:
# Copy original X features into a new DataFrame
X_interactions = X.copy()
# Define interaction terms
????

# Train a Poisson Regression model over all features
poisson_full_model_interactions = PoissonRegressor()
poisson_full_model_interactions.fit(
    ???,
    ???
)

In [ ]:
# Plot the original data per Ph Values along with the predictions of
# each of the 1d Poisson models,
# the full Poisson model
# and the full Poisson model with interactions
plt.figure(figsize=(10, 6))
for ph_value in X['pH'].unique():
    x_subset = X[X['pH'] == ph_value]['Biomasa']
    y_subset = y.loc[x_subset.index]
    
    x_vals = np.linspace(0, 10, 100).reshape(-1, 1)
    # Using ph_value as color
    # Truth
    plt.scatter(
        x_subset, y_subset,
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value}'
    )
    # 1D model
    plt.scatter(
        x_subset,
        poisson_1d_models[ph_value].predict(x_subset.values.reshape(-1, 1)),
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value} (1D model predicted)',
        alpha=0.75
    )
    plt.plot(
        x_vals,
        poisson_1d_models[ph_value].predict(x_vals),
        color=colors[ph_value],
        label=f'pH {ph_value} (1D model predicted)',
        alpha=0.75
    )

    # Full model
    X_subset = X.loc[x_subset.index]
    plt.scatter(
        x_subset,
        poisson_full_model.predict(X_subset),
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value} (Full model predicted)',
        alpha=0.5
    )
    plt.plot(
        x_vals,
        poisson_full_model.predict(
            np.concatenate(
                (np.full((x_vals.shape[0], 1), ph_value), x_vals),
                axis=1
            )
        ),
        color=colors[ph_value],
        # Different line style
        linestyle='--',
        label=f'pH {ph_value} (Full model predicted)',
        alpha=0.5
    )
    # Full model with interactions
    X_interactions_subset = X_interactions.loc[x_subset.index]
    plt.scatter(
        x_subset,
        poisson_full_model_interactions.predict(X_interactions_subset),
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value} (Full model with interactions predicted)',
        alpha=0.5
    )
    plt.plot(
        x_vals,
        poisson_full_model_interactions.predict(
            np.concatenate(
                (np.full((x_vals.shape[0], 1), ph_value), x_vals, x_vals * ph_value),
                axis=1
            )
        ),
        color=colors[ph_value],
        # Different line style
        linestyle=':',
        label=f'pH {ph_value} (Full model with interactions predicted)',
        alpha=0.5
    )
# Legend, outside the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Biomasa')
plt.ylabel('Especies')
plt.title('Poisson Regression Predictions, per pH value')
plt.show()

In [ ]:
# Replicate the above plot in log space:

# Plot the original data per Ph Values along with the predictions of
# each of the 1d Poisson models,
# the full Poisson model
# and the full Poisson model with interactions
plt.figure(figsize=(10, 6))
for ph_value in X['pH'].unique():
    x_subset = X[X['pH'] == ph_value]['Biomasa']
    y_subset = y.loc[x_subset.index]
    
    x_vals = np.linspace(0, 10, 100).reshape(-1, 1)
    # Using ph_value as color
    # Truth
    plt.scatter(
        x_subset,
        np.log(y_subset),
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value}'
    )
    # 1D model
    plt.scatter(
        x_subset,
        np.log(
            poisson_1d_models[ph_value].predict(x_subset.values.reshape(-1, 1))
        ),
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value} (1D model predicted)',
        alpha=0.75
    )
    plt.plot(
        x_vals,
        np.log(
            poisson_1d_models[ph_value].predict(x_vals)
        ),
        color=colors[ph_value],
        label=f'pH {ph_value} (1D model predicted)',
        alpha=0.75
    )

    # Full model
    X_subset = X.loc[x_subset.index]
    plt.scatter(
        x_subset,
        np.log(
            poisson_full_model.predict(X_subset)
        ),
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value} (Full model predicted)',
        alpha=0.5
    )
    plt.plot(
        x_vals,
        np.log(
                poisson_full_model.predict(
                np.concatenate(
                    (np.full((x_vals.shape[0], 1), ph_value), x_vals),
                    axis=1
                )
            )
        ),
        color=colors[ph_value],
        # Different line style
        linestyle='--',
        label=f'pH {ph_value} (Full model predicted)',
        alpha=0.5
    )
    # Full model with interactions
    X_interactions_subset = X_interactions.loc[x_subset.index]
    plt.scatter(
        x_subset,
        np.log(
            poisson_full_model_interactions.predict(X_interactions_subset)
        ),
        color=colors[ph_value],
        marker=markers[ph_value],
        label=f'pH {ph_value} (Full model with interactions predicted)',
        alpha=0.5
    )
    plt.plot(
        x_vals,
        np.log(
                poisson_full_model_interactions.predict(
                np.concatenate(
                    (np.full((x_vals.shape[0], 1), ph_value), x_vals, x_vals * ph_value),
                    axis=1
                )
            )
        ),
        color=colors[ph_value],
        # Different line style
        linestyle=':',
        label=f'pH {ph_value} (Full model with interactions predicted)',
        alpha=0.5
    )
# Legend, outside the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Biomasa')
plt.ylabel('Especies')
plt.title('Poisson Regression Predictions, per pH value')
plt.show()


In [ ]:
# Compare all poisson regression models

# Evaluate the 1d models
mae_1d = {}
mse_1d = {}
r2_1d = {}

for ph_value in X['pH'].unique():
    x_subset = X[X['pH'] == ph_value]['Biomasa']
    y_subset = y.loc[x_subset.index]
    y_pred_subset = poisson_1d_models[ph_value].predict(x_subset.values.reshape(-1, 1))
    mae_1d[ph_value] = mean_absolute_error(y_subset, y_pred_subset)
    mse_1d[ph_value] = mean_squared_error(y_subset, y_pred_subset)
    r2_1d[ph_value] = r2_score(y_subset, y_pred_subset)

# Evaluate the full model
y_pred_poisson = poisson_full_model.predict(X)
mae_full = mean_absolute_error(y, y_pred_poisson)
mse_full = mean_squared_error(y, y_pred_poisson)
r2_full = r2_score(y, y_pred_poisson)

# Evaluate the full model with interactions
y_pred_poisson_interactions = poisson_full_model_interactions.predict(X_interactions)
mae_full_interactions = mean_absolute_error(y, y_pred_poisson_interactions)
mse_full_interactions = mean_squared_error(y, y_pred_poisson_interactions)
r2_full_interactions = r2_score(y, y_pred_poisson_interactions)

# Print the results
print("\n--- Poisson Regression ---")
print("\n--------------------------")
print("1D Model Results:")
print("Mean Absolute Error: {}".format(
    np.mean(list(mae_1d.values())))
)
for ph_value, mae in mae_1d.items():
    print(f"\tpH {ph_value}: {mae}")
print("Mean Squared Error: {}".format(
    np.mean(list(mse_1d.values())))
)
for ph_value, mse in mse_1d.items():
    print(f"\tpH {ph_value}: {mse}")
print("R-squared: {}".format(
    np.mean(list(r2_1d.values())))
)
for ph_value, r2 in r2_1d.items():
    print(f"\tpH {ph_value}: {r2}")

print("\n--------------------------")
print("Full Model Results:")
print("Mean Absolute Error:", mae_full)
print("Mean Squared Error:", mse_full)
print("R-squared:", r2_full)

print("\n--------------------------")
print("Full Model with interactions Results:")
print("Mean Absolute Error:", mae_full_interactions)
print("Mean Squared Error:", mse_full_interactions)
print("R-squared:", r2_full_interactions)